In [0]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms

torch.backends.cudnn.deterministic=True
device = torch.device("cuda")

In [0]:
def generate_csv(test_predictions):
    
    csv_file  = open("/content/gdrive/My Drive/submission.csv", "w")
    
    csv_file.write("id,label")
    
    for i in range(len(test_predictions)):
        
        csv_file.write('\n')
        csv_file.write(str(i+1))
        csv_file.write(',')

        if test_predictions[i] == 0 :
            csv_file.write("Cat")   
            
        elif test_predictions[i] == 1:
            csv_file.write("Dog")
            
    csv_file.close()

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
X_test = np.load('/content/gdrive/My Drive/Datasets/CatsDogs/test.npy')
y_test = np.zeros(4999)

X_cats = np.load('/content/gdrive/My Drive/Datasets/CatsDogs/cats.npy')
y_cats = np.zeros(9999)

X_dogs = np.load('/content/gdrive/My Drive/Datasets/CatsDogs/dogs.npy')
y_dogs = np.ones(9999)

X_train = np.concatenate((X_cats, X_dogs))
y_train = np.concatenate((y_cats, y_dogs))

In [0]:
class CatDogDataset(Dataset):

    def __init__(self, X, y):
        
        self.X = X
        self.y = y
        
        self.len = (X.shape[0])
        
    def __getitem__(self, i):
        
        return torch.from_numpy(np.moveaxis(self.X[i], -1, 0)).type(torch.FloatTensor), int(self.y[i])

    def __len__(self):
        
        return self.len

In [0]:
test_dataset = CatDogDataset(X_test, y_test)
train_dataset = CatDogDataset(X_train, y_train)

In [0]:
train_loader = DataLoader(train_dataset, batch_size=66, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4999, shuffle=False)

In [0]:
class SmallVGG(nn.Module):
    
    
    def __init__(self):
        
        super().__init__()

        m = 2  
            
        # ~~ SmallVGGNET ~~ #
            
        # Convolutional block # 1   
            
        self.conv1 = nn.Conv2d(3, 16, 3, padding=3)
                
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
            
            
        # Convolutional block # 2 
            
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
            
            
        # Convolutional block # 3
            
        self.conv5 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv6 = nn.Conv2d(64, 64, 3, padding=1)
            
            
        # Fully connected block
            
        self.fc1 = nn.Linear(64*8*8, m)
        
        
    def forward(self, x):
        
        out = F.relu( self.conv1(x) )
        out = F.relu( self.conv2(out) )
        out = F.max_pool2d(out, 2)
            
        out = F.relu( self.conv3(out) )
        out = F.relu( self.conv4(out) )
        out = F.max_pool2d(out, 2)

        out = F.relu( self.conv5(out) )
        out = F.relu( self.conv6(out) )
        out = F.max_pool2d(out, 2)

        out = out.view(out.size(0), -1)

        out = self.fc1(out)
        
        return out

In [0]:
class Model:
    
    def __init__(self, architecture):
        
        self.net = architecture
    
    def train(self, train_loader, nb_epochs=30):
        
        #self.net.train()
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(self.net.parameters(), lr=0.005)
        
        train_accuracy = []
        train_er = []
        
        # Record train values
        train_accuracy_epoch, train_er_epoch = self.test(train_loader)
        
        print(train_er_epoch)
        print(train_accuracy_epoch)
            
        train_accuracy.append(train_accuracy_epoch)
        train_er.append(train_er_epoch)
        
        # Start training
        
        for epoch in range(nb_epochs):
            
            print('\nEpoch', epoch+1)
     
            for i, (X_batch, y_batch) in enumerate(train_loader):
        
                # Send the batch to the GPU
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                # Reset the gradients to zero
                optimizer.zero_grad()
        
                # Forward propagation
                y_hat_batch = self.net(X_batch)
        
                # Compute loss
                loss = criterion(y_hat_batch, y_batch)
            
                # Backward propagation
                loss.backward()
            
                # Update weights
                optimizer.step()
                
            # Record train values
            train_accuracy_epoch, train_er_epoch = self.test(train_loader)
            
            print(train_er_epoch)
            print(train_accuracy_epoch)
            
            train_accuracy.append(train_accuracy_epoch)
            train_er.append(train_er_epoch)
                                                       
        return train_accuracy, train_er
            
        
    def test(self, data_loader):
        
        #self.net.eval()
        
        criterion = nn.CrossEntropyLoss()
        
        with torch.no_grad():
        
            accuracy = 0
            correct_predictions = 0
            empirical_risk = 0
            
            for X_batch, y_batch in data_loader:
            
                # Send the batch to the GPU
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
                # Forward propagation
                y_hat_batch = self.net(X_batch)
                
                # Compute loss
                loss = criterion(y_hat_batch, y_batch)
            
                # Pick up most predicted class
                _, predictions = torch.max(y_hat_batch.data, 1)
                
                # Compare predictions and real label
                correct_predictions += (predictions == y_batch).sum().item()
                
                # Sum losses
                empirical_risk += loss  
        
        accuracy = (correct_predictions / len(data_loader.dataset)) * 100
        empirical_risk /= len(data_loader)
        
        return accuracy, empirical_risk
    
    
    def predict(self, data_loader):
        
        #self.net.eval()
        
        predictions = np.zeros( len(data_loader.dataset) )
        
        with torch.no_grad():
        
            for i, (X_batch, y_batch) in enumerate(data_loader):
            
                # Send the batch to the GPU
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
                y_hat_batch = self.net(X_batch)
            
                _, batch_predicictions = torch.max(y_hat_batch.data, 1)
                predictions[i * X_batch.shape[0] : (i+1) * X_batch.shape[0]] = batch_predicictions.cpu().numpy()
                
        return predictions

In [0]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

cnn = SmallVGG().to(device)

model  = Model(cnn)

train_accuracy, train_er = model.train(train_loader)

tensor(0.7172, device='cuda:0')
48.099809980998096

Epoch 1
tensor(0.6564, device='cuda:0')
61.431143114311425

Epoch 2
tensor(0.6325, device='cuda:0')
64.46644664466447

Epoch 3


In [0]:
test_predictions = model.predict(test_loader)

In [0]:
test_predictions.shape

In [0]:
generate_csv(test_predictions)